---

# Задание

В файлах *iCafe 2 month.xlsx* и *iCafe 15 days.xlsx* содержатся данные для обучения и тестирования классификаторов. Требуется построить три классификатора:

- На основе логистической регрессии;
- На основе дерева решений;
- На основе метода ближайшего соседа.

Основная цель - это достичь значение $F_1$ метрики на выборке 15 дней не менее чем:
- 0.86 для логистической регрессии;
- 0.9 для дерева решений;
- 0.89 для метода ближайшего соседа.

Вам будут даны небольшие подсказки в виде преобразования данных, но основные действия по преобразованию и выбору
признаков для обучения остаются за вами.

Так же, необходимо провести анализ и сделать выводы о результатах, при изменении параметров:
- ***max_depth***, при обучении дерева решений;
- ***n_neighbors***, при обучении классификатора методом ближайшего соседа.

Опишите эти выводы текстом в блокноте с решением. При каких значениях гиперпараметров модели становятся максимально переобученными? Какова область значений гиперпараметров, в которой они могут повлиять на качество обучения модели?

In [ ]:
from numpy.ma.core import append
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta, date, time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from IPython.core.display import Image, display
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from IPython.core.display import Image, display
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier
import pylab
import seaborn as sns
from datetime import datetime
plt.style.use('seaborn')
%matplotlib inline
pylab.rcParams['figure.figsize'] = (15, 10)
pd.set_option('display.max_columns', None)
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/MyDrive/pythonDS/Tables/iCafe 2 month.xlsx')
df_15 = pd.read_excel('/content/drive/MyDrive/pythonDS/Tables/iCafe 15 days.xlsx')
def day_of_the_week(df):
    df['day_of_the_week'] = [datetime.isoweekday(datetime.strptime(j, '%Y-%m-%d %H:%M:%S')) 
                             for j in df.Order_date_time]
    return df

df = day_of_the_week(df)
df_15 = day_of_the_week(df_15)

def share_date_time(df, sep = ' '):
    X = df.Order_date_time.str.split(pat = sep, expand = True)
    df['Order_date'], df['Order_time'] = X[0], X[1]
    return df
df_15 = share_date_time(df_15)
df = share_date_time(df)

def time_to_sec2(df):
    df['Order_time_sec'] = pd.to_timedelta(df.Order_time)//np.timedelta64(1,'s')
    lst = []
    for i in range(0,len(df_15.Rain)):
      if df_15.Rain[i] != 0.0 and df_15.Rain[i] != 1.0:
          lst.append(0)
      else:
        lst.append(df.Rain[i])
    df['Rain'] = lst
    return df
def time_to_sec(df):
    df['Order_time_sec'] = pd.to_timedelta(df.Order_time)//np.timedelta64(1,'s')
    return df
def Processing_results(sc, lr, df, target, attribute_name, name, add_schedule = []):
    X_std = sc.transform(df[attribute_name])
    new_col_name = []
    for i, col_name in enumerate(attribute_name):
        s = col_name + '_std_' + name
        df[s] = X_std[:,i]
        new_col_name.append(s)
    y_pred = 'y_pred_' + name
    Correct = 'Correct_' + name
    df[y_pred] = lr.predict(X_std)
    df[Correct] = df.Office == df[y_pred]
    print(classification_report(df.Office, df[y_pred]))
    return df

def log_reg(y,X):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
  sc = StandardScaler()
  sc.fit(X_train)                                  
  X_train_std = sc.transform(X_train)             
  X_test_std = sc.transform(X_test)                
  lr = LogisticRegression(random_state=21)
  lr.fit(X_train_std, y_train)
  y_pred = lr.predict(X_test_std)
  print(classification_report(y_test, y_pred))
  return sc,lr

def ran_for(y,X,m):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
  sc = StandardScaler()
  sc.fit(X_train)
  X_train_std = sc.transform(X_train)
  X_test_std = sc.transform(X_test)
  tree = DecisionTreeClassifier(random_state=21,max_depth=m)
  tree.fit(X_train_std, y_train)
  y_pred = tree.predict(X_test_std)
  return y_test, y_pred

def KNN(y,X,m):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
  knn = KNeighborsClassifier(n_neighbors=m)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  return y_test, y_pred

df_15 = time_to_sec2(df_15)
df = time_to_sec(df)
df_15['Order_sin'] = np.sin(df_15.Order_time_sec*math.pi/43200)
df_15['Order_cos'] = np.cos(df_15.Order_time_sec*math.pi/43200)
df['Order_sin'] = np.sin(df.Order_time_sec*math.pi/43200)
df['Order_cos'] = np.cos(df.Order_time_sec*math.pi/43200)
y = 'Office'
X = ['Order_sin', 'Order_cos', 'Delivery_time', 'Rain']
sc0,sc2 = ran_for(df_15[y], df_15[X],3)
sc3,sc4 = ran_for(df[y],df[X],7)
sc4,sc5 = KNN(df[y],df[X],5)
sc7,sc8 = KNN(df_15[y],df_15[X],5)
print('---------------------LOG_REG_TRAIN-------------------')
sc, lr = log_reg(df[y], df[X])
print('---------------------LOG_REG_TEST--------------------')
df_15 = Processing_results(sc,lr, df_15, y,X,'test', [X])
print('-----------------RANDOM_FOREST_TRAIN-----------------')
print(classification_report(sc0, sc2))
print('------------------RANDOM_FOREST_TEST-----------------')
print(classification_report(sc3, sc4))
print('----------------------KNN_TRAIN----------------------')
print(classification_report(sc7, sc8))
print('----------------------KNN_TEST-----------------------')
print(classification_report(sc4, sc5))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
---------------------LOG_REG_TRAIN-------------------
              precision    recall  f1-score   support

         0.0       0.86      0.85      0.85      2332
         1.0       0.88      0.88      0.88      2866

    accuracy                           0.87      5198
   macro avg       0.87      0.87      0.87      5198
weighted avg       0.87      0.87      0.87      5198

---------------------LOG_REG_TEST--------------------
              precision    recall  f1-score   support

         0.0       0.87      0.85      0.86      2767
         1.0       0.85      0.87      0.86      2706

    accuracy                           0.86      5473
   macro avg       0.86      0.86      0.86      5473
weighted avg       0.86      0.86      0.86      5473

-----------------RANDOM_FOREST_TRAIN-----------------
              precision    recall  f1-score   support

